In [0]:
import tensorflow as tf
tf.enable_eager_execution()
import tensorflow_datasets as tfds

import keras
from tensorflow.keras.layers import *
from keras.models import Model 
from keras.layers import Dropout
from keras.optimizers import SGD

W0815 14:24:36.391992 139992167126912 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Using TensorFlow backend.


In [0]:
#loading cifar10 data from tensorflow dataset
train_dataset = tfds.load("cifar10", with_info=False,split=tfds.Split.TRAIN)
train_dataset = train_dataset.map(lambda ele: tuple([ele['image'], ele['label']])).repeat().batch(16)

test_dataset = tfds.load("cifar10", with_info=False,split=tfds.Split.TEST)
test_dataset = test_dataset.map(lambda ele: tuple([ele['image'], ele['label']])).repeat().batch(16)


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


W0815 14:25:21.697309 139992167126912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


W0815 14:25:31.306321 139992167126912 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/1.0.2. Subsequent calls will reuse this data.


In [0]:
#modular inception blocks that are used a lot
#hyper parameters are reduced for cifar10 dataset rather than imagenet
def inception_block(x):
    x1 = Conv2D(filters=16,kernel_size=1,strides=2,padding='same',activation='relu')(x)
    x2 = Conv2D(filters=24,kernel_size=1,strides=2,padding='same',activation='relu')(x)
    x3 = Conv2D(filters=4,kernel_size=1,strides=2,padding='same',activation='relu')(x)
    x4 = MaxPooling2D(pool_size=3,strides=2,padding='same')(x)
    
    x2 = Conv2D(filters=32,kernel_size=3,strides=1,padding='same',activation='relu')(x2)
    #receptive field of 1x3 and 3x1 is equal to 3x3 receptive field but reduces parameters to speed up training 
    x2_1 = Conv2D(filters=32,kernel_size=(1,3),strides=1,padding='same',activation='relu')(x2)
    x2_2 = Conv2D(filters=32,kernel_size=(3,1),strides=1,padding='same',activation='relu')(x2)
    x3_1 = Conv2D(filters=8,kernel_size=(1,3),strides=1,padding='same',activation='relu')(x3)
    x3_2 = Conv2D(filters=8,kernel_size=(3,1),strides=1,padding='same',activation='relu')(x3)
    x4 = Conv2D(filters=8,kernel_size=1,strides=1,padding='same',activation='relu')(x4)
    
    con = concatenate([x1,x2_1,x2_2,x3_1,x3_2,x4])
    return con

In [0]:
inputs = Input(shape=(32,32,3))
x = Conv2D(filters=16,kernel_size=3,strides=2,padding='same',activation='relu')(inputs) #downsampling to 16x16
x = BatchNormalization()(x)

x = inception_block(x)

x = Conv2D(filters=32,kernel_size=3,strides=2,padding='same',activation='relu')(x) #downsampling to 8x8
x = BatchNormalization()(x)

x = inception_block(x)
x = inception_block(x)
x = inception_block(x)

x = Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu')(x) #downsampling to 4x4
x = BatchNormalization()(x)

x = inception_block(x)

x = AveragePooling2D(pool_size=(3,3),strides=2,padding='same')(x) #downsampling to 2x2
x = SpatialDropout2D(rate=0.3)(x) #dropout randomly takes out 3/10 of the weights, prevents overfitting in wider models like inception
x = Flatten()(x)
x = Dense(10, activation='relu')(x)
x = Softmax()(x)

model = tf.keras.Model(inputs,x)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 16, 16, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 16, 16, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 8, 8, 24)     408         batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
opt = tf.train.AdamOptimizer(learning_rate=0.0001) #defining learning rate
model.compile(optimizer=opt,loss='sparse_categorical_crossentropy',metrics=['accuracy']) #defining loss and metrics
model.fit(train_dataset,steps_per_epoch=50000,epochs=5,verbose=1,validation_data=test_dataset,validation_steps=10000)


Epoch 1/5


W0815 14:26:14.355024 139992167126912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


 4392/50000 [=>............................] - ETA: 30:39 - loss: 1.9526 - acc: 0.2841

In [0]:
image, label = next(iter(train_dataset)) #dividing data into x_train and y_train
image_val, label_val = next(iter(test_dataset)) #dividing data into x_val and y_val

In [0]:
#printing data to make sure data load worked

from matplotlib import pyplot as plt
%matplotlib inline

print('Training data shape', image.shape)
_,(ax1,ax2) = plt.subplots(1,2)
ax1.imshow(image_val[10])
ax2.imshow(image_val[3])

In [0]:
import numpy as np
answer = model.predict(next(iter(test_dataset)))
num_answer = np.argmax(answer[10])
#model's answer
num_answer

In [0]:
#actual answer in numpy
label_val[10]

In [0]:
#__AnswerKey__
# airplane: 0
# automobile: 1
# bird: 2
# cat: 3
# deer: 4
# dog: 5
# frog: 6
# horse: 7
# ship: 8
# truck: 9